In [1]:
import pandas as pd
import numpy as np

In [2]:
train = pd.read_csv('../../dataset/train.csv')
test = pd.read_csv('../../dataset/test.csv')
# 前処理を一度にやるためにtrainとtestをconcatする
test['Transported'] = np.nan
train_test = pd.concat([train, test], axis=0, ignore_index=True, sort=False)

# split on `/` to cols (deck/num/side)
def split_cabin(df):
    cabin = df['Cabin'].str.split('/', expand=True).rename(columns={0: 'CabinDeck', 1: 'CabinNum', 2: 'CabinSide'})
    cabin['CabinNum'] = cabin['CabinNum'].astype(float)
    return pd.concat([df, cabin], axis=1)

# group passenger or not
def make_group(df):
    df['GroupId'] = df['PassengerId'].apply(lambda x: x.split('_')[0])
    df['PeopleId'] = df['PassengerId'].apply(lambda x: x.split('_')[1])
    df['IsGroup'] = df['GroupId'].duplicated(keep=False)
    return df

# total room service, etc...
def total_bill(df):
    df['TotalBill'] = df[
        ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']].sum(axis=1)
    return df

# binalize 3 classes
def binalize_bill(df, th1=5000, th2=20000): 
    df = total_bill(df)
    df['BillBins'] = df['TotalBill'].apply(
        lambda x: 0 if x < th1 else (2 if x > th1 and x < th2 else 3))
    return df

def is_adult(df):
    df['IsAdult'] = df['Age'] >= 20
    return df

train_test = split_cabin(train_test)
train_test = make_group(train_test)
train_test = binalize_bill(train_test)
train_test = is_adult(train_test)

train_test.head()

/var/folders/zp/6qwnpvfn0cs2whczwk_5pvqh0000gs/T/ipykernel_36921/1980035355.py:5: FutureWarning: Behavior when concatenating bool-dtype and numeric-dtype arrays is deprecated; in a future version these will cast to object dtype (instead of coercing bools to numeric values). To retain the old behavior, explicitly cast bool-dtype arrays to numeric dtype.
  train_test = pd.concat([train, test], axis=0, ignore_index=True, sort=False)


,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,...,Transported,CabinDeck,CabinNum,CabinSide,GroupId,PeopleId,IsGroup,TotalBill,BillBins,IsAdult
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,...,0.0,B,0.0,P,0001,01,False,0.0,0,True
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,...,1.0,F,0.0,S,0002,01,False,736.0,0,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,...,0.0,A,0.0,S,0003,01,True,10383.0,2,True
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,...,0.0,A,0.0,S,0003,02,True,5176.0,2,True
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,...,1.0,F,1.0,S,0004,01,False,1091.0,0,False


### 使う特徴量を選ぶ

In [3]:
train_test = train_test[['HomePlanet', 'CryoSleep', 'Destination', 'Age', 'VIP', 'CabinDeck', 'CabinNum', 'CabinSide', 'IsGroup', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'TotalBill', 'BillBins', 'IsAdult', 'Transported']]
train_test.head()

,HomePlanet,CryoSleep,Destination,Age,VIP,CabinDeck,CabinNum,CabinSide,IsGroup,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,TotalBill,BillBins,IsAdult,Transported
0,Europa,False,TRAPPIST-1e,39.0,False,B,0.0,P,False,0.0,0.0,0.0,0.0,0.0,0.0,0,True,0.0
1,Earth,False,TRAPPIST-1e,24.0,False,F,0.0,S,False,109.0,9.0,25.0,549.0,44.0,736.0,0,True,1.0
2,Europa,False,TRAPPIST-1e,58.0,True,A,0.0,S,True,43.0,3576.0,0.0,6715.0,49.0,10383.0,2,True,0.0
3,Europa,False,TRAPPIST-1e,33.0,False,A,0.0,S,True,0.0,1283.0,371.0,3329.0,193.0,5176.0,2,True,0.0
4,Earth,False,TRAPPIST-1e,16.0,False,F,1.0,S,False,303.0,70.0,151.0,565.0,2.0,1091.0,0,False,1.0


### Encoding

In [4]:
# HomePlanet, Destination, CabinSideはonehot encoding
for col in ['HomePlanet', 'Destination', 'CabinDeck', 'CabinSide']:
    train_test = pd.concat([train_test,  pd.get_dummies(train_test[col], prefix=col)], axis=1)
    train_test = train_test.drop(col, axis=1)

In [5]:
# boolをintへ
def bool2int(df):
    for col in df.columns:
        if df[col].dtype == bool:
            df[col] = df[col].astype(float)
        if df[col].dtype == 'object':
            df[col] = df[col].map({True: 1, False: 0})
    return df

train_test = bool2int(train_test)

In [6]:
train_test

,CryoSleep,Age,VIP,CabinNum,IsGroup,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,...,CabinDeck_A,CabinDeck_B,CabinDeck_C,CabinDeck_D,CabinDeck_E,CabinDeck_F,CabinDeck_G,CabinDeck_T,CabinSide_P,CabinSide_S
0,0.0,39.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,1,0,0,0,0,0,0,1,0
1,0.0,24.0,0.0,0.0,0.0,109.0,9.0,25.0,549.0,44.0,...,0,0,0,0,0,1,0,0,0,1
2,0.0,58.0,1.0,0.0,1.0,43.0,3576.0,0.0,6715.0,49.0,...,1,0,0,0,0,0,0,0,0,1
3,0.0,33.0,0.0,0.0,1.0,0.0,1283.0,371.0,3329.0,193.0,...,1,0,0,0,0,0,0,0,0,1
4,0.0,16.0,0.0,1.0,0.0,303.0,70.0,151.0,565.0,2.0,...,0,0,0,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12965,1.0,34.0,0.0,1496.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,1,0,0,1
12966,0.0,42.0,0.0,NaN,0.0,0.0,847.0,17.0,10.0,144.0,...,0,0,0,0,0,0,0,0,0,0
12967,1.0,NaN,0.0,296.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,1,0,0,0,0,1,0
12968,0.0,NaN,0.0,297.0,0.0,0.0,2680.0,0.0,0.0,523.0,...,0,0,0,1,0,0,0,0,1,0


### モデリング

In [7]:
import lightgbm as lgbm

In [8]:
params = {
    'objective': 'binary',
    'num_leaves': 64,
    'min_child_samples': 20,
    'max_depth': 7,
    'n_estimators': 100,
    'learning_rate': 0.1,
    'importance_type': 'gain',
}

### 学習

In [9]:
import os
import random
from sklearn.model_selection import KFold, cross_validate
from sklearn.metrics import accuracy_score

In [10]:
def set_seed(seed=3407):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)


SEED = 3407
set_seed(SEED)

In [11]:
# DataFrameをnp.ndarrayに変換
trainval = train_test[~train_test['Transported'].isna()]
test = train_test[train_test['Transported'].isna()]
# inputとlabelに分離
x_trainval = trainval.drop('Transported', axis=1).values
y_trainval = trainval.Transported.values
x_test = test.drop('Transported', axis=1).values

In [12]:
x_trainval.shape, y_trainval.shape, x_test.shape

((8693, 29), (8693,), (4277, 29))

In [13]:
kf = KFold(n_splits=5, shuffle=True, random_state=SEED)

val_scores = []
models = []
for fold, (train_inds, val_inds) in enumerate(kf.split(x_trainval)):
    
    x_train, x_val = x_trainval[train_inds], x_trainval[val_inds]
    y_train, y_val = y_trainval[train_inds], y_trainval[val_inds]
    
    model = lgbm.LGBMClassifier(**params)
    model.fit(
        x_train, y_train,
        eval_set=(x_val, y_val),
        eval_metric='logloss',
        callbacks=[lgbm.early_stopping(10)],
    )

    y_val_pred = model.predict(x_val)
    score = accuracy_score(y_val, y_val_pred)
    print(f'fold {fold}/acc: {score}')
    val_scores.append(score)
    models.append(model)

cv_score = np.mean(val_scores)
print(f'CV score: {cv_score}')

Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[56]	valid_0's binary_logloss: 0.397179
fold 0/acc: 0.8027602070155262
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[77]	valid_0's binary_logloss: 0.381968
fold 1/acc: 0.8050603795284647
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[57]	valid_0's binary_logloss: 0.388475
fold 2/acc: 0.8136860264519838
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[73]	valid_0's binary_logloss: 0.368451
fold 3/acc: 0.8135788262370541
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[67]	valid_0's binary_logloss: 0.399908
fold 4/acc: 0.8003452243958573
CV score: 0.8070861327257772


In [14]:
importance = pd.DataFrame()

for i in range(len(models)):
    df = pd.DataFrame(models[i].feature_importances_,
                      index=trainval.columns[:-1], columns=[f'model{i+1}'])
    df = df.sort_values(f'model{i+1}', ascending=False)
    importance = pd.concat([importance, df], axis=1)

importance

,model1,model2,model3,model4,model5
TotalBill,8953.308842,9815.264425,9309.398884,9135.441295,9944.671035
FoodCourt,2758.468940,2561.863036,2466.704129,2705.293957,2757.269331
CabinNum,2308.927994,3033.579655,2429.537232,2757.470747,2689.105739
Transported,2127.913777,2031.203096,2067.694302,2080.129448,2019.216063
Spa,2070.264437,2261.312235,2057.465793,2149.559804,2200.079144
VRDeck,2002.107867,2015.679373,2010.374792,1987.463934,1977.760277
ShoppingMall,1932.063563,1920.111098,1837.528317,2025.967265,1849.161389
CryoSleep,1369.430899,1460.325848,1435.603274,1366.432717,1208.275833
Age,1266.974895,1346.694739,1376.091421,1365.081061,1372.162435
RoomService,1134.867343,1289.580050,1069.028497,1331.062456,1345.958434


### submit用のcsv作成

cvごとの推論の単純平均

In [15]:
y_preds = []

for i in range(len(models)):
    predictor = models[i]
    y_pred = predictor.predict_proba(x_test)
    y_preds.append(y_pred)

ensemble = np.argmax(np.mean(y_preds, axis=0), axis=-1)

In [16]:
test_ids = pd.read_csv('../../dataset/test.csv')['PassengerId']

df_submit = pd.DataFrame(ensemble, index=test_ids, columns=['Transported'])
df_submit.Transported = df_submit.Transported.astype(bool)

In [18]:
df_submit

,Transported
PassengerId,
0013_01,True
0018_01,False
0019_01,True
0021_01,True
0023_01,True
...,...
9266_02,True
9269_01,False
9271_01,True


In [17]:
df_submit.to_csv('submission/lgbm_trial7.csv')